# Collab Runtime Check VM and GPU

https://colab.research.google.com/notebooks/pro.ipynb#scrollTo=RFm2S0Gijqo8

In [ ]:
from google.colab import ai
response = ai.generate_text("What is the capital of France?")
print(response)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# Check Runtime

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Imports

In [ ]:
from google.colab import drive
import os

import torch
from transformers import BertTokenizerFast, BertModel
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

# Setup Drive

In [ ]:
drive.mount('/content/drive')

BASE_DIR = "/content/drive/MyDrive/ITRPA_PROJ"   # <– your project folder
PROCESSED_PATH = os.path.join(BASE_DIR, "processed_data.pkl")
DL_PATH = os.path.join(BASE_DIR, "dl_prepared_torch.pkl")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs_colab")

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Base:", BASE_DIR)
print("Outputs:", OUTPUT_DIR)


# Load Data

In [ ]:
with open(PROCESSED_PATH, "rb") as f:
    data = pickle.load(f)

X_train, X_test = data["X_train"], data["X_test"]
y_train, y_test = data["y_train_enc"], data["y_test_enc"]

print(f"Train: {len(X_train)} | Test: {len(X_test)}")
print("Example:", X_train[0][:120], "...")

# Initialize Bert

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased").to(device)
bert.eval();  # IMPORTANT: we’re not fine-tuning here

# LSTM

In [ ]:
@torch.no_grad()
def get_token_embeddings(texts, batch_size=8, max_len=96):
    outs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding="max_length", truncation=True,
                        max_length=max_len, return_tensors="pt")
        enc = {k: v.to(device) for k, v in enc.items()}
        last_hidden = bert(**enc).last_hidden_state   # [B, T, 768]
        outs.append(last_hidden.cpu())
    return torch.cat(outs, dim=0)                     # [N, T, 768]

X_train_tok = get_token_embeddings(X_train, batch_size=8, max_len=96)
X_test_tok  = get_token_embeddings(X_test,  batch_size=8, max_len=96)

print(X_train_tok.shape, X_test_tok.shape)  # e.g. torch.Size([N, 96, 768])

torch.save({
    "X_train_tok": X_train_tok,
    "X_test_tok": X_test_tok,
    "y_train": torch.tensor(y_train),
    "y_test": torch.tensor(y_test)
}, os.path.join(OUTPUT_DIR, "bert_token_emb.pt"))


In [ ]:
pack = torch.load(os.path.join(OUTPUT_DIR, "bert_token_emb.pt"))
Xtr, Xte = pack["X_train_tok"], pack["X_test_tok"]
ytr, yte = pack["y_train"], pack["y_test"]

train_ds = TensorDataset(Xtr, ytr)
test_ds  = TensorDataset(Xte, yte)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=16)

In [ ]:
class BertLSTMClassifier(nn.Module):
    def __init__(self, embed_dim=768, hidden_dim=128, num_classes=3, bidirectional=True, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(embed_dim, hidden_dim,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x):             # x: [B, T, 768]
        out, _ = self.lstm(x)         # [B, T, 2H]
        out = self.dropout(out[:, -1, :])  # last timestep
        return self.fc(out)

# Train Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = int(max(ytr.max().item(), yte.max().item()) + 1)

model = BertLSTMClassifier(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - loss: {total_loss/len(train_loader):.4f}")


# Evaluate Model

In [ ]:
model.eval()
correct = total = 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb).argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
print(f"BERT-LSTM Accuracy: {100*correct/total:.2f}%")


# Save Model

In [ ]:
torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "bert_lstm.pt"))
